# Indeed – Job Scraping

## Requirements 

In [124]:
# Lead Packages:
from bs4 import BeautifulSoup 
import requests
import numpy as np

In [173]:
def create_list_of_job_info_dict(job_listings, jobs=[]):
    """
    Function creates a list of job info dictionaries
    """
    
    jobs_list = []
    
    for job in job_listings:
        
        # create job info dict:
        job_info = dict()
        try:
            job_info['company'] = job.find("span", class_='company').text.strip()
        except AttributeError:
            job_info['company'] = None
            
        job_info['location'] = job.find('div', class_="recJobLoc")['data-rc-loc']
        job_info['title'] = job.find('a')['title'].strip()
        job_info['posted'] = job.find('span', class_='date').text
        job_info['description'] = job.find('div', class_='summary').text.strip().replace('\n', '')
        job_info['link'] = job.find('a')['href']
        
        # append dictionary to job list:
        jobs.append(job_info)
    
    return jobs
    
    

In [174]:
query = 'data'
location = 'israel'
days_ago = 2
url = f"https://il.indeed.com/jobs?q={query}&l={location}&fromage={days_ago}&sort=date" # add &start=1

In [175]:
url

'https://il.indeed.com/jobs?q=data&l=israel&fromage=2&sort=date'

In [176]:
webpage = requests.get(url)
soup = BeautifulSoup(webpage.content, "html.parser")

In [177]:
# how many jobs where found:
search_count = soup.find(id="resultsCol").find(id="searchCountPages").text
job_count = max([int(s) for s in search_count.replace(',', "").split() if s.isdigit()]) 
page_count = int(np.ceil(job_count // 15))

print(f"found {job_count} jobs")
print(f'{page_count} to scrape')

found 119 jobs
7 to scrape


In [187]:
jobs = []
for page in range(0, page_count*10+1, 10):
    url = f"https://il.indeed.com/jobs?q={query}&l={location}&fromage={days_ago}&start={page}" # add &start=1
    webpage = requests.get(url)
    soup = BeautifulSoup(webpage.content, "html.parser")
    job_listings = soup.find(id="resultsCol").find_all('div', class_='jobsearch-SerpJobCard')
    jobs = create_list_of_job_info_dict(job_listings, jobs)


In [186]:
len(jobs)

120

In [172]:

job_listings
for job in job_listings:
        print(job.find_all('span'))
        # create job info dict:
        job_info = dict()
        print(job.find('a')['href'])
        job_info['company'] = job.find("span", class_='company').text.strip()
        job_info['location'] = job.find('div', class_="recJobLoc")['data-rc-loc']
        job_info['title'] = job.find('a')['title'].strip()
        job_info['posted'] = job.find('span', class_='date').text
        job_info['description'] = job.find('div', class_='summary').text.strip().replace('\n', '')
        job_info['link'] = job.find('a')['href']
        
        # append dictionary to job list:
        print(job_info)

[<span class="new">חדש</span>, <span class="company">
Verint</span>, <span class="location accessible-contrast-color-location">הרצליה, מחוז תל אביב</span>, <span class="jobCardShelfIcon"><svg fill="none" height="16" viewbox="0 0 20 20" width="16"><rect fill="#FF5A1F" height="20" rx="10" width="20"></rect><path clip-rule="evenodd" d="M15.3125 4.0625L10.8125 15.3125L7.99999 11.375L15.3125 4.0625ZM7.604 12.7576L6.875 15.3125L8.567 14.1054L7.604 12.7576ZM7.20463 10.5796L12.419 5.36525L4.0625 9.125L6.9875 10.7968L7.20463 10.5796Z" fill="white" fill-rule="evenodd"></path></svg></span>, <span class="iaLabel iaIconActive">הגשת מועמדות בקלות</span>, <span class="date">פורסם כרגע</span>, <span class="tt_set" id="tt_set_0"><span class="result-link-bar-separator">·</span><a class="sl resultLink save-job-link" href="#" id="sj_bfcd90dde9cb5ffe" onclick="changeJobState('bfcd90dde9cb5ffe', 'save', 'linkbar', false, ''); return false;" title="שמירת משרה זו ל-my.indeed">שמירת משרה</a><span class="result

AttributeError: 'NoneType' object has no attribute 'text'